<a href="https://colab.research.google.com/github/nityambhargava/Amazon_Clone/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [4]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Data Collection - Kaggle API**

In [5]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [6]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [7]:
#setup kaggle credentials as environment varaiables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [1]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 89% 23.0M/25.7M [00:01<00:00, 26.2MB/s]
100% 25.7M/25.7M [00:01<00:00, 16.2MB/s]


In [8]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [9]:
#unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [10]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


**Loading the dataset**

In [11]:
data = pd.read_csv("IMDB Dataset.csv")

In [12]:
data.shape

(50000, 2)

In [13]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [14]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [15]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [16]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [17]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [18]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [19]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [20]:
# split cell into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [21]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Data Preprocessing**

In [22]:
#Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [23]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [24]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [25]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [26]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


**LSTM - Long Short Term Memory**

In [28]:
#build the model

model  = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid")) #output layer

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 128)          640000    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
#compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

**Training the model**

In [32]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 [==============================] - 396s 787ms/step - loss: 0.4073 - accuracy: 0.8105 - val_loss: 0.3234 - val_accuracy: 0.8621
Epoch 2/5
500/500 [==============================] - 385s 769ms/step - loss: 0.2711 - accuracy: 0.8918 - val_loss: 0.2920 - val_accuracy: 0.8802
Epoch 3/5
500/500 [==============================] - 371s 743ms/step - loss: 0.2245 - accuracy: 0.9117 - val_loss: 0.2962 - val_accuracy: 0.8754
Epoch 4/5
500/500 [==============================] - 367s 734ms/step - loss: 0.1876 - accuracy: 0.9273 - val_loss: 0.3132 - val_accuracy: 0.8794
Epoch 5/5
500/500 [==============================] - 368s 736ms/step - loss: 0.1579 - accuracy: 0.9396 - val_loss: 0.3544 - val_accuracy: 0.8741


In [36]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 [==============================] - 26s 84ms/step - loss: 0.3294 - accuracy: 0.8805
Test Loss: 0.32938286662101746
Test Accuracy: 0.8805000185966492


**Building a Predictive system**

In [39]:
def predict_sentiment(review):
  #tokenize and pad the review
  sequence= tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "Positive" if prediction [0][0] > 0.5 else "Negative"
  return sentiment

In [41]:
#example usage
new_review = "This movie was amazing. Fell in love with the storyline!"
sentiment = predict_sentiment(new_review)
print(f"The Sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 82ms/step
The Sentiment of the review is: Positive


In [42]:
#example usage
new_review = "This movie was not that good!"
sentiment = predict_sentiment(new_review)
print(f"The Sentimentof the review is: {sentiment}")

1/1 [==============================] - 0s 161ms/step
The Sentimentof the review is: Negative
